<a href="https://colab.research.google.com/github/gabrielutomo/Youtube-Music-Hits/blob/main/Naive%20Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT LIBRARY

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt

#MEMBACA DATASET

In [ ]:
# Load dataset
data = pd.read_csv("/content/youtube-top-100-songs-2025 (3).csv")

# Lihat 5 data pertama
display(data.head())

# Cek struktur data
data.info()


In [ ]:
# Import pandas
import pandas as pd

# Load dataset
data = pd.read_csv("/content/youtube-top-100-songs-2025 (3).csv")

# Lihat 5 data pertama
display(data.head())

# Cek struktur data melihat struktur dataset, seperti title, categories, dan apakah ada data kosong.”
data.info()
# Dari 100 data, hanya 85 video punya tags,15 video tidak memiliki tag

#CEK APAKAH ADA DATA YANG NULL DAN DUPLIKAT

In [ ]:
# Cek data kosong (null)
print("=== CEK DATA NULL ===")
print(data.isnull().sum())

# Cek duplikat
print("\nJumlah duplikat:", data.duplicated().sum())

# Hapus duplikat jika ada
data = data.drop_duplicates()
# mengecek apakah dataset memiliki data kosong pada setiap kolom.

#MEMISAHKAN FITUR DAN LABEL

In [ ]:
from sklearn.preprocessing import LabelEncoder
# X = variabel input (fitur) yang digunakan untuk memprediksi sesuatu
# y = label atau target yang ingin diprediksi
# Tentukan fitur dan label
from sklearn.preprocessing import LabelEncoder

# Copy data agar aman
df = data.copy()

# Buat encoder
le_categories = LabelEncoder()
le_tags = LabelEncoder()
le_channel = LabelEncoder()

# Encode fitur kategorikal
df['categories_encoded'] = le_categories.fit_transform(df['categories'])
df['tags_encoded'] = le_tags.fit_transform(df['tags'].astype(str))  # pastikan string
df['channel_encoded'] = le_channel.fit_transform(df['channel'])

# Tentukan fitur dan label
X = df[['categories_encoded', 'duration', 'tags_encoded', 'channel_encoded']]
y = df['view_count']  # asli, tidak di-encode

print("Contoh fitur:")
print(X.head())

print("\nContoh label (view_count asli):")
print(y.head())
# Ini adalah data input (X) yang akan digunakan model untuk belajar memprediksi view_count.

#BALANCING DATA MENGGUNAKAN SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Cek distribusi sebelum SMOTE
# LabelEncoder Mengubah view_count menjadi kelas karena nanti dipakai SMOTE (SMOTE hanya bisa untuk classification)
print("=== Distribusi Sebelum Balancing ===")
print(pd.Series(y_encoded).value_counts())

# Handle missing values in 'tags' (This line caused the KeyError and is removed)
# X['tags'].fillna('', inplace=True)

# Encode categorical features
categorical_features = ['categories', 'tags', 'channel']
one_hot = OneHotEncoder(handle_unknown='ignore')
transformer = ColumnTransformer([('one_hot', one_hot, categorical_features)],
                                  remainder='passthrough')

# For X_encoded, we need to use the original df for OneHotEncoder, not X (which is already label encoded)
# The X passed to transformer.fit_transform should contain the raw categorical features for OneHotEncoder.
# Let's reconstruct X for this step, based on df, before applying OneHotEncoder.

# Recreate X with original categorical columns for OneHotEncoder
X_for_onehot = df[['categories', 'duration', 'tags', 'channel']].copy()
# Fill NA in tags for X_for_onehot before OneHotEncoding if desired
X_for_onehot['tags'].fillna('', inplace=True)

X_encoded = transformer.fit_transform(X_for_onehot)


# Terapkan SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_encoded, y_encoded)

# Cek distribusi setelah SMOTE (Membuat kelas menjadi seimbang dengan menambah data sintetis)
print("\n=== Distribusi Setelah SMOTE ===")
print(pd.Series(y_resampled).value_counts())

# SMOTE tidak mengubah apa pun.
# Distribusi label tetap sama persis, jumlahnya tetap 100 kelas dengan frekuensi 1.



#NORMALISASI DAN SPLIT DATA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Normalisasi fitur agar skala serupa
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X_resampled)

# Bagi data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_resampled, test_size=0.2, random_state=42
)

print("Data training:", X_train.shape)
print("Data testing:", X_test.shape)

# membagi data menjadi data latih (train) dan data uji (test)

#TRAINING MODEL NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Buat model Naive Bayes Gaussian Digunakan untuk memprediksi berdasarkan distribusi Gaussian (normal).
model = GaussianNB()

# Latih model
# Convert sparse matrix to dense array
model.fit(X_train.toarray(), y_train)

print("Model berhasil dilatih ✅")

#EVALUASI MODEL

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np # Import numpy for unique

# Prediksi data uji Model Naive Bayes yang sudah dilatih digunakan untuk memprediksi label dari data test.
y_pred = model.predict(X_test.toarray())

# Menggabungkan semua label asli (y_test) dan label prediksi (y_pred).
unique_labels = np.unique(np.concatenate((y_test, y_pred)))

# Di bagian ini, label angka (hasil encoding) dikembalikan ke nilai view_count asli, supaya confusion matrix labelnya terbaca (angka view_count asli)
target_names_subset = le.inverse_transform(unique_labels).astype(str).tolist()


# Evaluasi hasil Menghitung persentase prediksi yang benar.
print("=== HASIL EVALUASI ===")
print(f"Akurasi: {accuracy_score(y_test, y_pred)*100:.2f}%\n")
print("Classification Report:\n", classification_report(y_test, y_pred, labels=unique_labels, target_names=target_names_subset, zero_division=0))

# Confusion Matrix visual
cm = confusion_matrix(y_test, y_pred, labels=unique_labels)
plt.figure(figsize=(8, 6)) # Increase figure size for better readability
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names_subset, yticklabels=target_names_subset)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

#PREDIKSI MANUAL (INPUT DARI USER)

In [ ]:
import pandas as pd

# Input manual - ask for the correct features and handle types
categories_input = input("Masukkan kategori (contoh: Music): ")
duration_input = float(input("Masukkan durasi dalam detik (contoh: 173): "))
tags_input = input("Masukkan tags (pisahkan dengan semicolon, contoh: Pop;Rock): ")
channel_input = input("Masukkan nama channel (contoh: ROSÉ): ")


# Create a DataFrame with the same column names as the training data
input_df = pd.DataFrame([[categories_input, duration_input, tags_input, channel_input]],
                        columns=['categories', 'duration', 'tags', 'channel'])

# Apply the same preprocessing as used for the training data
# We need to use the fitted preprocessor and scaler
input_processed = preprocessor.transform(input_df)
input_scaled = scaler.transform(input_processed)


# Prediksi
pred = model.predict(input_scaled.toarray()) # Convert to dense array for GaussianNB
pred_label = le.inverse_transform(pred)

print(f"\nPrediksi View Count: {pred_label[0]}")